# အစည်းအဝေး ၁ – Chat Bootstrap (Foundry Local)

ဒီ notebook က Foundry Local ကို bootstrap လုပ်ပြီး၊ အကြိုက်ဆုံး model alias ကို download လုပ်ကာ၊ ပုံမှန် chat completion နဲ့ streaming chat completion နှစ်မျိုးလုံးကို ဆောင်ရွက်ပေးပါမယ်။


# အခြေအနေ
ဒီအစည်းအဝေးမှာ Foundry Local ကို အသုံးပြုပြီး ဒေသတွင်း အငယ်စား ဘာသာစကားမော်ဒယ်တစ်ခုကို တုံ့ပြန်စေဖို့ လိုအပ်တဲ့ အနည်းဆုံးအချက်တွေကို မိတ်ဆက်ပေးမှာဖြစ်ပါတယ်။ သင်လုပ်ဆောင်ရမယ့်အရာတွေကတော့:
- SDK / client အခြေခံလိုအပ်ချက်တွေကို ထည့်သွင်းမည်။
- ရွေးချယ်ထားတဲ့ alias (ပုံမှန်: `phi-3.5-mini`) အတွက် Foundry Local manager ကို စတင်မည်။
- မော်ဒယ် metadata ထဲမှာ optional fields တွေကို ခံနိုင်ရည်ရှိအောင် defensive monkey‑patch ကို အသုံးပြုမည်။
- ပုံမှန် chat completion request တစ်ခုကို ပို့မည်။
- တုံ့ပြန်မှုကို token‑by‑token အနေနဲ့ စီးဆင်းကြည့်မည်။

ဒီလုပ်ငန်းစဉ်ရဲ့ ရည်ရွယ်ချက်ကတော့ RAG, routing, ဒါမှမဟုတ် agents တွေဆီ ရောက်မီမှာ သင့်ဒေသတွင်း runtime နဲ့ network path ကို အတည်ပြုဖို့ ဖြစ်ပါတယ်။


### ရှင်းလင်းချက်: Dependency Installation
ဤအနည်းဆုံး chat flow အတွက်လိုအပ်သော Python packages များကို install လုပ်ပါသည်။
- `foundry-local-sdk`: Local models နှင့် service lifecycle ကိုစီမံရန်။
- `openai`: Chat completions အတွက်အသုံးပြုရလွယ်သော client abstraction။
- `rich`: Notebook output ကိုပိုမိုရှင်းလင်းစေရန် pretty printing။

ပြန်လည် run လုပ်ခြင်းသည်လုံခြုံပြီး (idempotent) ဖြစ်သည်။ သင့်ပတ်ဝန်းကျင်တွင်အဆိုပါ packages များရှိပြီးသားဖြစ်ပါက skip လုပ်နိုင်သည်။


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### ရှင်းလင်းချက် - အဓိက Import များ
Notebook တစ်ခုလုံးတွင် အသုံးပြုမည့် module များကို ယူဆောင်လာသည်။
- `FoundryLocalManager` သည် ဒေသခံ model runtime နှင့် ဆက်သွယ်ရန်။
- `OpenAI` client သည် ရင်းနှီးပြီးသား chat completion API ကို ပြန်လည်အသုံးပြုနိုင်ရန်။
- `rich.print` သည် အလှဆင် output များအတွက်။

ဤနေရာတွင် network call မရှိပါ—ဤသည် namespace ကို ပြင်ဆင်ခြင်းသာဖြစ်သည်။


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### ရှင်းလင်းချက် - Manager Initialization & Metadata Patch
`FoundryLocalManager` ကို ရွေးချယ်ထားသော alias အတွက် initialize လုပ်ပြီး `promptTemplate` က null ဖြစ်နိုင်သော service response များကို သေချာစွာ ကိုင်တွယ်နိုင်ရန်အတွက် ကာကွယ်ရေး monkey-patch တစ်ခုကို အသုံးပြုသည်။

အဓိကရလဒ်များ:
- Service state နှင့် endpoint ကို အတည်ပြုသည်။
- Cached models များကို စာရင်းပြုစုသည် (local store ကို အတည်ပြုသည်)။
- Alias အတွက် concrete model ID ကို ရှင်းလင်းသည် (နောက်ပိုင်း chat calls များတွင် အသုံးပြုရန်)။

Raw service metadata တွင် validation ပြဿနာများကို တွေ့ရှိပါက၊ SDK ကို fork မလုပ်ဘဲ sanitize လုပ်နည်းကို ဒီ pattern မှာ ပြသထားသည်။


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### ရှင်းလင်းချက် - အခြေခံ Chat Completion
`OpenAI`-နှင့်သင့်လျော်သော client ကို ဒေသခံ Foundry endpoint ကိုညွှန်းပြီး single non-streaming chat completion ကိုလုပ်ဆောင်ပါသည်။ အဓိကအချက်များမှာ:
- မော်ဒယ်သည် အမှားမဖြစ်ဘဲ တုံ့ပြန်မှုပေးနိုင်ရန် သေချာပါစေ။
- Latency / output format ကိုအတည်ပြုပါ။
- အရင်းအမြစ်များကိုထိန်းသိမ်းရန် `max_tokens` ကိုသင့်တော်အောင်ထားပါ။

ဤလုပ်ဆောင်မှုမအောင်မြင်ပါက Foundry Local service သည် run ဖြစ်နေကြောင်းနှင့် alias သေချာစွာအလုပ်လုပ်နေကြောင်း ပြန်လည်စစ်ဆေးပါ။


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### ရှင်းလင်းချက်: Streaming Chat Completion
Token streaming ကို အသုံးပြု၍ latency ကို လျော့ချပြီး interactive UX ကို တိုးတက်စေခြင်းကို ပြသထားသည်။ loop သည် ရောက်ရှိလာသော incremental deltas များကို ပုံမှန်ပုံနှိပ်ပြသသည်။
- အစောပိုင်း partial output အရေးကြီးသော chat UI များအတွက် အသုံးဝင်သည်။
- Token throughput နှင့် full completion latency ကို တိုင်းတာနိုင်စေသည်။

ဤ pattern ကို tokens များစုစည်းခြင်း၊ progress widget ကို update ပြုလုပ်ခြင်း၊ သို့မဟုတ် mid-generation တွင် abort ပြုလုပ်ခြင်းအတွက် အလွယ်တကူ ပြောင်းလဲအသုံးပြုနိုင်သည်။


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေပါသော်လည်း အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအမှားများ သို့မဟုတ် အနားယူမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
